
# Emotion Detector (4 Classes) — Keras + EfficientNetB0

**Classes:** `angry`, `happy`, `sad`, `surprised`  
**Goal:** ≥ 90% accuracy on your test set.  
**Run order:** top → bottom, one cell at a time.

> If you're on Windows, use raw strings like `r"C:\\Users\\you\\Downloads\\ML_Projects\\Emotion Detector"` for paths.


## 0) Install/Check libraries

In [4]:
!pip install --upgrade tensorflow


  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=f061bb737fe56431df675f9be7e0a61035a5c717711725e277b698c2051a56a1
  Stored in directory: c:\users\punya\appdata\local\pip\cache\wheels\22\4c\94\0583f60c9c5b6024ed64f290cb2d43b06bb4f75577dc3c93a7
Successfully built clang
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.6.0
    Uninstalling google-auth-2.6.0:
      Successfully uninstalled google-auth-2.6.0
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.36.1
    Uninstalling grpcio-1.36.1:
      Successfully uninstalled grpcio-1.36.1
  Attempting uninstall: tensorboar

In [1]:

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# EDIT ONLY THESE THREE LINES to your actual folders
BASE_DIR = r"C:\Users\punya\Downloads\ML_Projects\Emotion Detector"  # <-- CHANGE THIS
TRAIN_DIR = BASE_DIR + r"\train"
TEST_DIR = BASE_DIR + r"\test"
SINGLE_TEST_DIR = BASE_DIR + r"\Single_TEST"
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

import os, json, math, random
import numpy as np
from pathlib import Path

print("TRAIN_DIR:", TRAIN_DIR)
print("TEST_DIR :", TEST_DIR)
print("SINGLE_TEST_DIR:", SINGLE_TEST_DIR)

assert os.path.isdir(TRAIN_DIR), "TRAIN_DIR does not exist"
assert os.path.isdir(TEST_DIR), "TEST_DIR does not exist"
assert os.path.isdir(SINGLE_TEST_DIR), "SINGLE_TEST_DIR does not exist"


TRAIN_DIR: C:\Users\punya\Downloads\ML_Projects\Emotion Detector\train
TEST_DIR : C:\Users\punya\Downloads\ML_Projects\Emotion Detector\test
SINGLE_TEST_DIR: C:\Users\punya\Downloads\ML_Projects\Emotion Detector\Single_TEST


## 2) Config

In [18]:

SEED = 42
IMG_SIZE = (224, 224)         # EfficientNetB0 default
BATCH = 32
VAL_SPLIT = 0.1
EPOCHS_STAGE1 = 15
EPOCHS_STAGE2 = 15
BASE_LR = 1e-3
FINETUNE_LR = 5e-5
TARGET_CLASSES = ["angry", "happy", "sad", "surprised"]
MODEL_PATH = str(Path(BASE_DIR) / "emotion_effnetb0.h5")
LABELS_PATH = str(Path(BASE_DIR) / "labels_map.json")

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
print("Paths OK. Will save model to:", MODEL_PATH)


Paths OK. Will save model to: C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


## 3) Sanity check: folder structure & counts

In [19]:

from collections import Counter
def count_images(root):
    total = 0
    per_class = {}
    for cls in sorted(os.listdir(root)):
        cls_path = os.path.join(root, cls)
        if os.path.isdir(cls_path):
            n = sum([1 for f in os.listdir(cls_path) if f.lower().endswith(('.jpg','.jpeg','.png'))])
            per_class[cls] = n
            total += n
    return total, per_class

train_total, train_counts = count_images(TRAIN_DIR)
test_total, test_counts = count_images(TEST_DIR)

print("TRAIN total:", train_total, train_counts)
print("TEST  total:", test_total, test_counts)

# Check class names match expectation
assert set(train_counts.keys()) == set(TARGET_CLASSES), f"Train classes {train_counts.keys()} != {TARGET_CLASSES}"
assert set(test_counts.keys()) == set(TARGET_CLASSES), f"Test classes {test_counts.keys()} != {TARGET_CLASSES}"


TRAIN total: 19211 {'angry': 3995, 'happy': 7215, 'sad': 4830, 'surprised': 3171}
TEST  total: 4810 {'angry': 958, 'happy': 1774, 'sad': 1247, 'surprised': 831}


## 4) Data pipeline (augmentation + validation split)

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input as rn_preprocess
preprocess_input = rn_preprocess  # ensure we use ResNet normalization

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VAL_SPLIT,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VAL_SPLIT
)

train_gen = train_datagen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    seed=SEED,
    subset='training'
)

valid_gen = valid_datagen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False,
    seed=SEED,
    subset='validation'
)

print("Class indices:", train_gen.class_indices)


Found 17291 images belonging to 4 classes.
Found 1920 images belonging to 4 classes.
Class indices: {'angry': 0, 'happy': 1, 'sad': 2, 'surprised': 3}


## 5) Handle class imbalance (class weights)

In [21]:

cls_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weights = {i: w for i, w in enumerate(cls_weights)}
print("Class weights:", class_weights)


Class weights: {0: np.float64(1.202099555061179), 1: np.float64(0.6656529103788112), 2: np.float64(0.994421440073614), 3: np.float64(1.5146285914505957)}


## 6) Build model (EfficientNetB0 + small head)

In [22]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as rn_preprocess
preprocess_input = rn_preprocess  # use this in your generators

tf.keras.backend.clear_session()
inp = keras.Input(shape=(224,224,3))
base = ResNet50(include_top=False, weights='imagenet', input_tensor=inp)
base.trainable = False
x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
out = layers.Dense(4, activation='softmax')(x)
model = keras.Model(inp, out)
model.compile(keras.optimizers.Adam(learning_rate=BASE_LR), 'categorical_crossentropy', ['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,114,308 (91.99 MB)

 Trainable params: 526,084 (2.01 MB)

 Non-trainable params: 23,588,224 (89.98 MB)

## 7) Train — Stage 1 (frozen backbone)

In [24]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1),
    keras.callbacks.ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1)
]

steps_per_epoch = int(np.ceil(train_gen.samples / BATCH))
val_steps = int(np.ceil(valid_gen.samples / BATCH))

history1 = model.fit(
    train_gen,
    epochs=EPOCHS_STAGE1,          # e.g., 15
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_gen,
    validation_steps=val_steps,
    class_weight=class_weights,    # from cell 6
    verbose=1,
    callbacks=callbacks
)


C:\Users\punya\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4242 - loss: 1.4621  
Epoch 1: val_accuracy improved from None to 0.56719, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 669s 1s/step - accuracy: 0.4554 - loss: 1.2928 - val_accuracy: 0.5672 - val_loss: 1.0055 - learning_rate: 0.0010
Epoch 2/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 971ms/step - accuracy: 0.5077 - loss: 1.1347  
Epoch 2: val_accuracy improved from 0.56719 to 0.58125, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 583s 1s/step - accuracy: 0.5143 - loss: 1.1169 - val_accuracy: 0.5813 - val_loss: 0.9962 - learning_rate: 0.0010
Epoch 3/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 972ms/step - accuracy: 0.5293 - loss: 1.0804  
Epoch 3: val_accuracy did not improve from 0.58125
541/541 ━━━━━━━━━━━━━━━━━━━━ 583s 1s/step - accuracy: 0.5267 - loss: 1.0855 - val_accuracy: 0.5750 - val_loss: 1.0080 - learning_rate: 0.0010
Epoch 4/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 972ms/step - accuracy: 0.5323 - loss: 1.0692  
Epoch 4: val_accuracy improved from 0.58125 to 0.60521, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 583s 1s/step - accuracy: 0.5345 - loss: 1.0747 - val_accuracy: 0.6052 - val_loss: 0.9535 - learning_rate: 0.0010
Epoch 5/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 973ms/step - accuracy: 0.5348 - loss: 1.0678  
Epoch 5: val_accuracy did not improve from 0.60521
541/541 ━━━━━━━━━━━━━━━━━━━━ 584s 1s/step - accuracy: 0.5385 - loss: 1.0572 - val_accuracy: 0.6016 - val_loss: 0.9616 - learning_rate: 0.0010
Epoch 6/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 971ms/step - accuracy: 0.5385 - loss: 1.0580  
Epoch 6: val_accuracy improved from 0.60521 to 0.63177, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 583s 1s/step - accuracy: 0.5452 - loss: 1.0543 - val_accuracy: 0.6318 - val_loss: 0.9081 - learning_rate: 0.0010
Epoch 7/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 972ms/step - accuracy: 0.5473 - loss: 1.0458  
Epoch 7: val_accuracy did not improve from 0.63177
541/541 ━━━━━━━━━━━━━━━━━━━━ 583s 1s/step - accuracy: 0.5477 - loss: 1.0496 - val_accuracy: 0.6057 - val_loss: 0.9238 - learning_rate: 0.0010
Epoch 8/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 970ms/step - accuracy: 0.5507 - loss: 1.0450  
Epoch 8: val_accuracy did not improve from 0.63177
541/541 ━━━━━━━━━━━━━━━━━━━━ 582s 1s/step - accuracy: 0.5488 - loss: 1.0463 - val_accuracy: 0.6187 - val_loss: 0.9010 - learning_rate: 0.0010
Epoch 9/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 972ms/step - accuracy: 0.5540 - loss: 1.0398  
Epoch 9: val_accuracy did not improve from 0.63177
541/541 ━━━━━━━━━━━━━━━━━━━━ 579s 1s/step - accuracy: 0.5507 - loss: 1.0390 - val_accuracy: 0.6047 - val_loss: 0.9435 - learning_rate: 0.0010
Epoch 10/

## 8) Fine-tune — Unfreeze top layers

In [25]:

num_layers = len(base.layers)
unfreeze_from = int(num_layers * 0.6)  # unfreeze top ~40%
for i, layer in enumerate(base.layers):
    layer.trainable = (i >= unfreeze_from)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=FINETUNE_LR),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history2 = model.fit(
    train_gen,
    epochs=EPOCHS_STAGE2,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_gen,
    validation_steps=val_steps,
    class_weight=class_weights,
    verbose=1,
    callbacks=callbacks
)


Epoch 1/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6101 - loss: 0.9440   
Epoch 1: val_accuracy improved from 0.63177 to 0.74948, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 1027s 2s/step - accuracy: 0.6595 - loss: 0.8377 - val_accuracy: 0.7495 - val_loss: 0.6587 - learning_rate: 5.0000e-05
Epoch 2/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7422 - loss: 0.6644   
Epoch 2: val_accuracy improved from 0.74948 to 0.76875, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 1013s 2s/step - accuracy: 0.7453 - loss: 0.6581 - val_accuracy: 0.7688 - val_loss: 0.6054 - learning_rate: 5.0000e-05
Epoch 3/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7741 - loss: 0.5885   
Epoch 3: val_accuracy did not improve from 0.76875
541/541 ━━━━━━━━━━━━━━━━━━━━ 1010s 2s/step - accuracy: 0.7752 - loss: 0.5820 - val_accuracy: 0.7255 - val_loss: 0.7068 - learning_rate: 5.0000e-05
Epoch 4/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7965 - loss: 0.5368   
Epoch 4: val_accuracy improved from 0.76875 to 0.77292, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 1067s 2s/step - accuracy: 0.7959 - loss: 0.5353 - val_accuracy: 0.7729 - val_loss: 0.5824 - learning_rate: 5.0000e-05
Epoch 5/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8123 - loss: 0.4938   
Epoch 5: val_accuracy improved from 0.77292 to 0.79740, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 1025s 2s/step - accuracy: 0.8104 - loss: 0.4948 - val_accuracy: 0.7974 - val_loss: 0.5134 - learning_rate: 5.0000e-05
Epoch 6/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8321 - loss: 0.4494   
Epoch 6: val_accuracy improved from 0.79740 to 0.81250, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 1027s 2s/step - accuracy: 0.8275 - loss: 0.4606 - val_accuracy: 0.8125 - val_loss: 0.5165 - learning_rate: 5.0000e-05
Epoch 7/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8402 - loss: 0.4275   
Epoch 7: val_accuracy improved from 0.81250 to 0.81510, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 1026s 2s/step - accuracy: 0.8375 - loss: 0.4319 - val_accuracy: 0.8151 - val_loss: 0.4967 - learning_rate: 5.0000e-05
Epoch 8/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8532 - loss: 0.3913   
Epoch 8: val_accuracy did not improve from 0.81510
541/541 ━━━━━━━━━━━━━━━━━━━━ 1025s 2s/step - accuracy: 0.8503 - loss: 0.3947 - val_accuracy: 0.7953 - val_loss: 0.5599 - learning_rate: 5.0000e-05
Epoch 9/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8692 - loss: 0.3537   
Epoch 9: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.

Epoch 9: val_accuracy improved from 0.81510 to 0.81615, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 1027s 2s/step - accuracy: 0.8601 - loss: 0.3728 - val_accuracy: 0.8161 - val_loss: 0.5421 - learning_rate: 5.0000e-05
Epoch 10/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8867 - loss: 0.3131   
Epoch 10: val_accuracy improved from 0.81615 to 0.82344, saving model to C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5


541/541 ━━━━━━━━━━━━━━━━━━━━ 1027s 2s/step - accuracy: 0.8883 - loss: 0.3060 - val_accuracy: 0.8234 - val_loss: 0.5243 - learning_rate: 2.5000e-05
Epoch 11/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8953 - loss: 0.2807   
Epoch 11: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.

Epoch 11: val_accuracy did not improve from 0.82344
541/541 ━━━━━━━━━━━━━━━━━━━━ 1030s 2s/step - accuracy: 0.8994 - loss: 0.2760 - val_accuracy: 0.8224 - val_loss: 0.5255 - learning_rate: 2.5000e-05
Epoch 12/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9136 - loss: 0.2354   
Epoch 12: val_accuracy did not improve from 0.82344
541/541 ━━━━━━━━━━━━━━━━━━━━ 1116s 2s/step - accuracy: 0.9160 - loss: 0.2326 - val_accuracy: 0.8219 - val_loss: 0.5300 - learning_rate: 1.2500e-05
Epoch 13/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9203 - loss: 0.2221   
Epoch 13: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.

Epoch 13: val_accuracy improved f

541/541 ━━━━━━━━━━━━━━━━━━━━ 1062s 2s/step - accuracy: 0.9208 - loss: 0.2240 - val_accuracy: 0.8333 - val_loss: 0.5348 - learning_rate: 1.2500e-05
Epoch 14/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9299 - loss: 0.1995   
Epoch 14: val_accuracy did not improve from 0.83333
541/541 ━━━━━━━━━━━━━━━━━━━━ 1099s 2s/step - accuracy: 0.9307 - loss: 0.1959 - val_accuracy: 0.8323 - val_loss: 0.5311 - learning_rate: 6.2500e-06
Epoch 15/15
541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9367 - loss: 0.1817   
Epoch 15: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-06.

Epoch 15: val_accuracy did not improve from 0.83333
541/541 ━━━━━━━━━━━━━━━━━━━━ 1137s 2s/step - accuracy: 0.9351 - loss: 0.1876 - val_accuracy: 0.8240 - val_loss: 0.5295 - learning_rate: 6.2500e-06


## 9) Evaluate on TEST set

In [4]:
# Self-contained test evaluation cell for local Jupyter
import os, sys, glob, numpy as np
from importlib import import_module

# --- Imports (safe even if some were already imported) ---
try:
    from tensorflow import keras
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
except Exception as e:
    raise RuntimeError("TensorFlow imports failed. Make sure tensorflow is installed.") from e

from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image

# --- Try to use existing notebook variables; fall back to sensible defaults/search ---
# We'll not overwrite variables if they exist in the notebook's global namespace.
g = globals()

# TEST_DIR
if 'TEST_DIR' in g and os.path.isdir(g['TEST_DIR']):
    TEST_DIR = g['TEST_DIR']
else:
    # try common places: current dir / test, ./data/test, ../data/test
    candidates = ['test', './test', './data/test', './dataset/test']
    found = None
    for c in candidates:
        if os.path.isdir(c):
            found = os.path.abspath(c); break
    if found is None:
        # try any folder that contains the 4 class names
        for root, dirs, files in os.walk('.', topdown=True):
            lower_dirs = [d.lower() for d in dirs]
            if set(['angry','happy','sad','surprised']).issubset(set(lower_dirs)):
                found = os.path.abspath(root); break
    if found is None:
        raise FileNotFoundError("Couldn't locate TEST_DIR automatically. Define TEST_DIR variable to your local test folder (containing class subfolders).")
    TEST_DIR = found

# IMG_SIZE
if 'IMG_SIZE' in g:
    IMG_SIZE = g['IMG_SIZE']
else:
    IMG_SIZE = (224, 224)

# BATCH
if 'BATCH' in g:
    BATCH = g['BATCH']
else:
    BATCH = 32

# MODEL_PATH: prefer variable, else try to find .h5 in cwd or parent
if 'MODEL_PATH' in g and os.path.isfile(g['MODEL_PATH']):
    MODEL_PATH = g['MODEL_PATH']
else:
    # search for likely model filenames
    patterns = ['**/*emotion*.h5', '**/*best*.h5', '**/*.h5']
    found_model = None
    for pat in patterns:
        res = glob.glob(pat, recursive=True)
        if res:
            # prefer files in current working dir or a Drive-like path; pick newest
            res = sorted(res, key=os.path.getmtime, reverse=True)
            found_model = res[0]; break
    if found_model is None:
        raise FileNotFoundError("No .h5 model file found. Either set MODEL_PATH variable to your saved model or place the .h5 in this working directory.")
    MODEL_PATH = os.path.abspath(found_model)

# preprocess_input: try common backbones' preprocessors if not defined
if 'preprocess_input' in g:
    preprocess_input = g['preprocess_input']
else:
    # try ResNet50 preprocessor, then MobileNetV2
    try:
        from tensorflow.keras.applications.resnet50 import preprocess_input as _p
        preprocess_input = _p
    except Exception:
        try:
            from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as _p
            preprocess_input = _p
        except Exception:
            # fallback: scale by 1/255
            def preprocess_input(x):
                return x / 255.0

# --- Report what will be used ---
print("Using TEST_DIR =", TEST_DIR)
print("Using MODEL_PATH =", MODEL_PATH)
print("IMG_SIZE =", IMG_SIZE, "BATCH =", BATCH)
print("Using preprocess_input from:", getattr(preprocess_input, "__module__", "fallback (scaled/255)"))

# --- Build non-augmented test generator ---
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    directory=TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False
)

# --- Load model and evaluate ---
print("\nLoading model...")
best_model = keras.models.load_model(MODEL_PATH)
print("Model loaded. Running prediction on test set (this may take a bit)...")

probs = best_model.predict(test_gen, verbose=1)
y_pred = np.argmax(probs, axis=1)
y_true = test_gen.classes

idx_to_class = {v: k for k, v in test_gen.class_indices.items()}
print("\nClassification report (TEST):")
print(classification_report(y_true, y_pred, target_names=[idx_to_class[i] for i in range(len(idx_to_class))]))

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix (rows=true, cols=pred):\n", cm)
test_acc = (y_true == y_pred).mean()
print(f"\nTEST accuracy: {test_acc*100:.2f}%")


Using TEST_DIR = C:\Users\punya\Downloads\ML_Projects\Emotion Detector\test
Using MODEL_PATH = C:\Users\punya\Downloads\ML_Projects\Emotion Detector\emotion_effnetb0.h5
IMG_SIZE = (224, 224) BATCH = 32
Using preprocess_input from: keras.src.applications.resnet
Found 4810 images belonging to 4 classes.

Loading model...


C:\Users\punya\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Model loaded. Running prediction on test set (this may take a bit)...
151/151 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step

Classification report (TEST):
              precision    recall  f1-score   support

       angry       0.74      0.69      0.71       958
       happy       0.90      0.91      0.91      1774
         sad       0.77      0.76      0.77      1247
   surprised       0.83      0.89      0.86       831

    accuracy                           0.83      4810
   macro avg       0.81      0.81      0.81      4810
weighted avg       0.82      0.83      0.82      4810

Confusion matrix (rows=true, cols=pred):
 [[ 659   53  196   50]
 [  40 1623   56   55]
 [ 165   91  948   43]
 [  29   34   26  742]]

TEST accuracy: 82.58%


## 10) Predict on `Single_TEST` images

In [8]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications.resnet50 import preprocess_input

# 1) Recreate labels_map from TEST_DIR
classes = sorted(os.listdir(TEST_DIR))
labels_map = {i: cls for i, cls in enumerate(classes)}
print("labels_map:", labels_map)

# 2) Function to load & preprocess
def load_and_prep(img_path):
    img = Image.open(img_path).convert('RGB').resize(IMG_SIZE)
    arr = np.array(img).astype(np.float32)
    arr = preprocess_input(arr)
    arr = np.expand_dims(arr, axis=0)
    return arr

# 3) Collect images from Single_TEST
image_files = [
    os.path.join(SINGLE_TEST_DIR, f)
    for f in os.listdir(SINGLE_TEST_DIR)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
]

# 4) Run prediction
if not image_files:
    print("No images found in Single_TEST.")
else:
    for p in sorted(image_files):
        arr = load_and_prep(p)
        pr = best_model.predict(arr, verbose=0)[0]
        pred_idx = int(np.argmax(pr))
        pred_label = labels_map[pred_idx]
        conf = float(np.max(pr))
        print(f"{os.path.basename(p)} -> {pred_label} ({conf*100:.1f}% confidence)")


labels_map: {0: 'angry', 1: 'happy', 2: 'sad', 3: 'surprised'}
angry.png -> sad (74.4% confidence)
angry2.png -> angry (66.5% confidence)
happy.png -> happy (99.6% confidence)
happy2.jpg -> sad (43.2% confidence)
sad.png -> sad (99.5% confidence)
surprised.png -> surprised (99.6% confidence)


## 11) Save model + labels map

In [11]:
import json, os

# Define BASE_DIR if not defined
BASE_DIR = r"C:\Your\Project\Folder"  # <-- change this

# Build labels map from TEST folder
classes = sorted(os.listdir(TEST_DIR))
labels_map = {i: cls for i, cls in enumerate(classes)}

# Save model (HDF5 or native Keras)
MODEL_PATH = os.path.join(BASE_DIR, "emotion_model.h5")       # HDF5
MODEL_PATH_KERAS = os.path.join(BASE_DIR, "emotion_model.keras")  # native

best_model.save(MODEL_PATH)          # HDF5 version
best_model.save(MODEL_PATH_KERAS)    # new recommended version

# Save labels
LABELS_PATH = os.path.join(BASE_DIR, "labels_map.json")
with open(LABELS_PATH, "w") as f:
    json.dump(labels_map, f)

print("Saved:")
print("  Model:", MODEL_PATH)
print("  Model:", MODEL_PATH_KERAS)
print("  Labels:", LABELS_PATH)


Saved:
  Model: C:\Your\Project\Folder\emotion_model.h5
  Model: C:\Your\Project\Folder\emotion_model.keras
  Labels: C:\Your\Project\Folder\labels_map.json
